## Kaggle House Price Competition 

# Cleyton Candeira

Real estate sales in the United States 🏡🇺🇸 can be influenced by various factors, including economic bargaining, which often plays a significant role in consumers' buying decisions. Instead of solely focusing on the physical features of a property, such as the number of bedrooms and bathrooms, buyers may also consider economic factors like the price, interest rates (not relevant in this context 😊), and the expected property appreciation.

During the real estate boom in the USA, primarily in the early 2000s, there was a substantial increase in demand for properties, resulting in a considerable rise in home prices 📈🏠. During this period, many people viewed property purchases as a profitable investment, and some even transitioned into becoming real estate agents, capitalizing on the expanding market 📊💰.

*Informative Addition*

However, this real estate bubble was unsustainable, and in 2008, the subprime crisis occurred. This crisis was triggered by the collapse of the high-risk mortgage market (known as subprime mortgages), leading to mass borrower defaults and widespread property devaluation 📉🏚️. The real estate market collapsed, resulting in a significant economic recession in the USA and worldwide 🌍📉.

This financial crisis of 2008 had a lasting impact on the real estate sector and the overall economy, serving as a reminder of the risks associated with the real estate market and uncontrolled speculation. Since then, there have been significant changes in financial and mortgage regulations to prevent similar crises in the future 🏦🔒.



* The challenge seeks to find the best price prediction model for properties in the city of Ames, Iowa.

**I also took some inspiration and used some techniques of other people, but I did not hard-copied, he said**

# Data Treatment

In [ ]:
#Basic Packages 

import sys
import scipy

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
#!pip install numpy
import seaborn as sns


print('Environment specification:\n')
print('python', '%s.%s.%s' % sys.version_info[:3])

for mod in np, scipy, sns, pd:
    print(mod.__name__, mod.__version__)

In [ ]:
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

df_all = pd.concat([df_train, df_test], sort = True, ignore_index = True)

df_train.head(15)

In [ ]:
#Let's check data type

def count_data_types(df):
    data_types = df.dtypes
    count = data_types.value_counts()
    return count

result = count_data_types(df_all)

#Show results dtypes
print(result)

## SalesPrice Treatment

In [ ]:
# Use the .describe() method to generate descriptive statistics
description = df_train['SalePrice'].describe()

# Print the descriptive statistics
print(description)

# Create a figure with two subplots: boxplot and histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Boxplot
sns.set(style="whitegrid")
sns.boxplot(x=df_train['SalePrice'], ax=axes[0])
axes[0].set_title('SalePrice Distribution (Boxplot)', fontsize=16)
axes[0].set_xlabel('SalePrice', fontsize=14)
axes[0].tick_params(axis='x', labelsize=12)
axes[0].tick_params(axis='y', labelsize=12)

# Histogram
sns.histplot(df_train['SalePrice'], ax=axes[1], kde=True, color='skyblue')
axes[1].set_title('SalePrice Distribution (Histogram)', fontsize=16)
axes[1].set_xlabel('SalePrice', fontsize=14)
axes[1].tick_params(axis='x', labelsize=12)
axes[1].tick_params(axis='y', labelsize=12)

plt.tight_layout()
plt.show()


**As it exhibits a non-normal probability distribution, it's crucial to conduct variable transformation tests to assess the attainment of a more 'normalized' distribution** 📊

We will test three different transformations (square root, logarithm, and Box-Cox) on the 'SalePrice' variable, and then it will display a boxplot alongside a histogram for each of them. Additionally, it will perform a normality test (e.g., Shapiro-Wilk test) after each transformation and show the p-value. The transformation that results in a p-value closer to 1 will indicate a more normal distribution. 📊🔄📈


In [ ]:
# Assuming 'df' is your DataFrame with the 'SalePrice' variable

# Define a list of transformation names and corresponding functions
transformations = {
    'Original': lambda x: x,
    'Square Root': np.sqrt,
    'Logarithm': np.log1p
}

# Prepare a color palette for plotting
colors = sns.color_palette('Set2', n_colors=len(transformations))

# Create subplots for boxplots and histograms
fig, axes = plt.subplots(nrows=len(transformations), ncols=2, figsize=(12, 8))

for i, (transformation_name, transformation_func) in enumerate(transformations.items()):
    # Apply the transformation to 'SalePrice'
    transformed_price = transformation_func(df_train['SalePrice'])
    
    # Plot the boxplot
    sns.boxplot(x=transformed_price, ax=axes[i, 0], color=colors[i])
    axes[i, 0].set_title(f'Boxplot - {transformation_name}')
    
    # Plot the histogram
    sns.histplot(transformed_price, ax=axes[i, 1], kde=True, color=colors[i])
    axes[i, 1].set_title(f'Histogram - {transformation_name}')
    
    # Perform a normality test (Shapiro-Wilk) and display the p-value
    p_value = stats.shapiro(transformed_price)[1]
    axes[i, 1].text(0.6, 0.8, f'p-value: {p_value:.4f}', transform=axes[i, 1].transAxes)

plt.tight_layout()
plt.show()


**As you can observe, the most effective transformation is the logarithmic one. It successfully normalizes the probability distribution of SalePrice in the best possible manner**

In [ ]:
SalePrice = np.log1p(df_train["SalePrice"])
df_train["SalePrice"] = SalePrice

not_nan_indices = df_all['SalePrice'].notna()
df_all.loc[not_nan_indices, 'SalePrice'] = SalePrice

## Cleaning Data 🧹

In [ ]:
def clean_data(df):
    # Check and clean whitespace in strings
    string_columns = df.select_dtypes(include=['object']).columns
    for col in string_columns:
        df[col] = df[col].str.strip()  # Remove leading and trailing whitespaces

    # Check and replace commas with periods in numeric values
    numeric_columns = df.select_dtypes(include=['float', 'int']).columns
    for col in numeric_columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace(',', '.', regex=True).astype(float)
    
    # Other checks and cleanups can be added here

    return df

In [ ]:
df_all = clean_data(df_all)

## Check NaN values 

In [ ]:
# Calculate the percentage of NaN values in each column
#sales = df_all['SalePrice']

#df_all.drop(columns='SalePrice', inplace=True)

nan_percentage = (df_all.isna().mean() * 100).sort_values(ascending=False)

# Filter columns with NaN values
nan_columns = nan_percentage[nan_percentage > 0.1]

# Create a bar plot to visualize the NaN percentages for selected columns
plt.figure(figsize=(12, 6))
sns.barplot(x=nan_columns, y=nan_columns.index, palette="viridis")
plt.xlabel('% of NaN Values')
plt.ylabel('Columns')
plt.title('Percentage of NaN Values per Column')
plt.show()

# Display columns with NaN values and their percentages
print("Columns with NaN values and their percentages:")
print(nan_columns)

**There are many variables with missing values, what to do?! Well, we can start from the simplest to the most complex. Categorical variables can be predicted through a DecisionTrees classification model. Can it be tiring? Maybe, but we'll start with those variables where there are less than 20% missing values, which are:**
- LotFrontage
- GarageYrBlt
- GarageCond
- GarageType
- GarageFinish
- GarageQual
...

**However, we cannot accurately predict missing values without a comprehensive understanding of the relationships between all variables, including their weights, correlations, and other relevant factors. So we'll first have to do an EDA of the House Pricing database...**

## EDA 📊🔍

In [ ]:
#Remove Id Col
if 'Id' in df_all.columns:
    df_all.drop(['Id'], axis=1, inplace=True)
    
# Check if these columns are categorical or numeric
qualitative = [col for col in df_all.columns if df_all[col].dtype == 'object']
quantitative = [col for col in df_all.columns if df_all[col].dtype != 'object']

#Remove Id && SalePrice
if 'SalePrice' in quantitative:
    quantitative.remove('SalePrice')

**There are many qualitative and quantitative columns in the dataframe. We need to identify the most important columns that influence SalePrice before addressing missing values. This can save time and effort**

### Qualitative Analysis

First, we must to split Categorical Nominal cols and Categorical Ordinal cols. In this section, we'll focus on feature engineering, inspired by a fantastic Kaggle script by [Ryan Holbrook](https://www.kaggle.com/code/ryanholbrook/feature-engineering-for-house-prices). Thank you, Ryan, for your wonderful script!


In [ ]:
# Determine which variables are ordinal and which are nominal
ordinal_variables = [
    "OverallQual", "OverallCond", "ExterQual", "ExterCond", "BsmtQual", "BsmtCond",
    "HeatingQC", "KitchenQual", "FireplaceQu", "GarageQual", "GarageCond", "PoolQC",
    "LotShape", "LandSlope", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "Functional",
    "GarageFinish", "PavedDrive", "Utilities", "CentralAir", "Electrical", "Fence"
]

nominal_variables = [col for col in qualitative if col not in ordinal_variables]

# Fill NaN values in ordinal and nominal columns with "Missing_Value"
for col in ordinal_variables + nominal_variables:
    df_all[col].fillna("Missing_Value", inplace=True)
    df_train[col].fillna("Missing_Value", inplace=True)

    
# Convert ordinal categorical variables to ordinal data type
for col in ordinal_variables:
    df_all[col] = pd.Categorical(df_all[col], ordered=True)
    df_train[col] = pd.Categorical(df_train[col], ordered=True)

# Note that MSSubClass is being recognized as a numeric variable due to its configuration.

# Convert nominal categorical variables to nominal data type
for col in nominal_variables:
    df_all[col] = pd.Categorical(df_all[col], ordered=False)
    df_train[col] = pd.Categorical(df_train[col], ordered=False)

In [ ]:
# Create a loop to generate multiple boxplots

# Number of boxplots per row
boxplots_per_row = 2

# Calculate the number of rows required
num_rows = (len(qualitative) + boxplots_per_row - 1) // boxplots_per_row

# Loop to generate the boxplots
for i, col in enumerate(qualitative):
    if i % boxplots_per_row == 0:
        # Create a new figure at the beginning of each row
        plt.figure(figsize=(12, 6))  # Adjust the figure size as needed

    plt.subplot(1, boxplots_per_row, i % boxplots_per_row + 1)
    sns.boxplot(x=col, y='SalePrice', data=df_train)
    plt.xticks(rotation=90)
    plt.title(f'Boxplot of SalePrice vs {col}')

    if (i + 1) % boxplots_per_row == 0 or i == len(qualitative) - 1:
        # If it's the last plot in the row, or a new row begins
        plt.tight_layout()
        plt.show()

In [ ]:
# Create a loop to generate multiple plots
plot_rows = len(quantitative)
plot_cols = 2  # Two plots per row

# Create a list of warm colors
warm_colors = ['#FF5733', '#FFC300', '#FF5733', '#FFC300', '#FF5733', '#FFC300', '#FF5733']

# Create a DataFrame to store normality test results
normality_results = pd.DataFrame(columns=['Variable', 'P-Value', 'Is Normal'])

# Initialize the figure and axes outside the loop
fig, axes = plt.subplots(nrows=plot_rows, ncols=plot_cols, figsize=(12, 4 * plot_rows))

# Loop to generate the plots and perform normality tests
for i, col in enumerate(quantitative):
    row = i // plot_cols
    col_idx = i % plot_cols

    # Use the warm color from the list
    sns.histplot(df_all[col], kde=True, ax=axes[row, col_idx], color=warm_colors[i % len(warm_colors)])
    axes[row, col_idx].set_title(f'Distribution of {col}')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')

    # Perform a normality test (Shapiro-Wilk)
    _, p_value = stats.shapiro(df_all[col])
    alpha = 0.05  # Significance level
    is_normal = bool(p_value > alpha)  # Explicitly cast to bool dtype

    # Add the results to the DataFrame
    normality_results.loc[len(normality_results)] = [col, p_value, is_normal]

# Remove empty subplots
if len(quantitative) < plot_rows * plot_cols:
    for i in range(len(quantitative), plot_rows * plot_cols):
        fig.delaxes(axes.flatten()[i])

plt.tight_layout()
plt.show()

# Print the normality test results DataFrame
print(normality_results)

#### Definition of ANOVA (Analysis of Variance)

An F-test is any statistical test in which the test statistic has an F-distribution under the null hypothesis. It is most often used when comparing statistical models that have been fitted to a data set, in order to identify the model that best fits the population from which the data were sampled. Exact "F-tests" mainly arise when the models have been fitted to the data using least squares. The name was coined by George W. Snedecor, in honour of Ronald Fisher. Fisher initially developed the statistic as the variance ratio in the 1920s. In summary, Analysis of Variance (ANOVA) is a statistical technique used to compare the means of three or more groups or categories. It is often used to test whether there are significant differences between the population means of distinct groups. It provides a structured approach to hypothesis testing and allows for deeper insights into group differences. However, it's crucial to use ANOVA appropriately, considering its underlying assumptions and conducting post-hoc tests when necessary, to draw valid conclusions from your data.

#### Mathematical Formulation


Let's assume you have $(k)$ groups with $(n_i)$ observations in each group, where $(i = 1, 2, ..., k)$.

The statistical hypotheses are formulated as:

- **Null Hypothesis $(H_0)$:** All population means are equal.
  
  $H_0: \mu_1 = \mu_2 = \ldots = \mu_k$

- **Alternative Hypothesis $(H_1)$:** At least one population mean is different from the others.
  
  $H_1: \text{At least one of the population means is different.}$

The F-test statistic is calculated as the ratio of between-group variability to within-group variability. The formula for the F-statistic is:

$F = \frac{\text{Between-Group Variability}}{\text{Within-Group Variability}}$

Where:

- **Between-Group Variability** is the sum of squared differences between the means of each group and the overall mean, weighted by the number of observations in each group.
  $\text{Between-Group Variability} = \sum_{i=1}^{k} n_i (\bar{X}_i - \bar{X})^2$
  
- **Within-Group Variability** is the sum of squared differences between each observation and the mean of the group to which it belongs.
  
  $\text{Within-Group Variability} = \sum_{i=1}^{k} \sum_{j=1}^{n_i} (X_{ij} - \bar{X}_i)^2$

Here, $(\bar{X}_i)$ represents the mean of group $(i)$, and $(\bar{X})$ is the overall mean.

Finally, the F-statistic follows an F-distribution with $(k-1)$ and $(N - k)$ degrees of freedom, where $(N)$ is the total number of observations.


**Reference:**
- [ANOVA - Wikipedia](https://en.wikipedia.org/wiki/Analysis_of_variance)





In [ ]:
# Define a function to perform ANOVA
def perform_anova(frame):
    # Create an empty DataFrame to store results
    anova_results = pd.DataFrame()
    anova_results['Feature'] = qualitative  # Assuming qualitative is predefined

    p_values = []

    # Iterate through qualitative features
    for feature in qualitative:
        # Create a list to store samples for each category
        samples = []

        # Iterate through unique categories within the feature
        for category in frame[feature].unique():
            # Get SalePrice values for the current category
            sale_prices = frame[frame[feature] == category]['SalePrice'].values
            samples.append(sale_prices)

        # Perform ANOVA test and get the p-value
        p_value = stats.f_oneway(*samples)[1]
        p_values.append(p_value)

    # Add p-values to the DataFrame
    anova_results['p-value'] = p_values

    # Sort the results by p-value
    anova_results = anova_results.sort_values(by='p-value')

    return anova_results

# Perform ANOVA on the 'train' DataFrame
result = perform_anova(df_train)

# Calculate the disparity for visualization
result['Disparity'] = np.log(1.0 / result['p-value'].values)

# Create a barplot to visualize the results
plt.figure(figsize=(12, 6))
sns.barplot(data=result, x='Feature', y='Disparity')
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Disparity (log(1/p-value))')
plt.title('ANOVA Test Results')
plt.show()

In [ ]:
# Sort 'result' by the top 20 disparities
result_top_25 = result.sort_values(by='Disparity', ascending=False).head(25)

# Get the names of variables from the top 20 results
top_25_features = result_top_25['Feature'].tolist()

# Find the variables that match the names in 'top_20_features'
matching_variables = nan_columns[nan_columns.index.isin(top_25_features)]

cols_to_fix = list(matching_variables.index)

sns.set(style="whitegrid")

# Crie um gráfico de barras usando seaborn
plt.figure(figsize=(12, 6))  # Ajuste o tamanho do gráfico conforme necessário
sns.barplot(x=matching_variables.index, y=matching_variables.values, palette="viridis")

# Rotacione os rótulos do eixo x para facilitar a leitura
plt.xticks(rotation=90)

# Adicione rótulos e título
plt.xlabel('Variáveis Correspondentes')
plt.ylabel('Valores')
plt.title('Variáveis Correspondentes entre "nan_columns" e Top 25 de "result"')

# Ajuste automaticamente o layout para evitar cortar rótulos
plt.tight_layout()

# Exiba o gráfico
plt.show()


**Categorical Variables to Address Missing Values for SalePrice Prediction:**

   - FireplaceQu
   - GarageCond
   - GarageType
   - GarageFinish
   - GarageQual
   - BsmtExposure
   - BsmtQual
   - BsmtCond
   - BsmtFinType1
   - MasVnrType
   - Electrical

### Quantitative

#### Correlation (Spearman, Pearson, and Kendall)

Spearman, Kendall, and Pearson are correlation coefficients used to measure the relationship between variables. Spearman is ideal for assessing monotonic relationships in ordinal data or when a non-linear relationship is suspected, as it focuses on rank orders. Kendall's Tau is also suitable for ordinal data and assesses the concordance of rankings, making it robust to outliers and non-linear relationships. Pearson is best suited for linear relationships between continuous variables, assuming a normal distribution. It's sensitive to outliers and may not capture non-linear associations effectively. The choice depends on the nature of the data and the type of relationship under investigation, with Spearman and Kendall being preferable when data is not normally distributed or non-linearity is expected.

**Spearman Rank Correlation:**

*Mathematical Definition:*

The Spearman rank correlation coefficient, denoted by $ρs$, is calculated as the correlation between the rankings (orders) of variables $X$ and $Y$. It is given by the formula:

$\rho_s = 1 - \frac{6\sum{d_i^2}}{n(n^2 - 1)}$

Where:
- $d_i$ is the difference between the ranks of $X$ and $Y$ for the $i-th$ observation.
- $n$ is the number of observations.

**Pearson Correlation Coefficient:**

*Mathematical Definition:* 

The Pearson correlation coefficient, denoted by $\rho$ (rho), measures the linear correlation between two continuous variables $XX$ and $YY$. It is given by the formula:

   $\rho = \frac{\sum{(X - \bar{X})(Y - \bar{Y})}}{\sqrt{\sum{(X - \bar{X})^2} \sum{(Y - \bar{Y})^2}}}$

Where:
- $\bar{X}$ is the mean of $X$.
- $\bar{Y}$ is the mean of $Y$..

**Kendall Correlation Coefficient (Kendall's Tau):**

*Mathematical Definition:* 

The Kendall correlation coefficient, denoted by $\tau$ (tau), quantifies the agreement or disagreement in the rankings of pairs of observations between two variables. It is computed as:

$\tau = \frac{{\text{number of concordant pairs} - \text{number of discordant pair}}}{{\frac{1}{2}n(n-1)}}$

Where:
- The number of concordant pairs represents pairs of observations with the same relative order in both variables.
- The number of discordant pairs represents pairs of observations with differing relative orders in both variables.
- n is the total number of observations.

These mathematical definitions elucidate the calculation process for each correlation coefficient, conveying the statistical relationship between variables $X$ and $Y$ in terms of rankings, linearity, or concordance, as applicable to each coefficient.

The chosen correlation is Spearman because it assumes a monotonic relationship between the independent variables and the SalePrice variable. This also means that their ranking order will be taken into account. 

In [ ]:
# Calculating Spearman correlations between quantitative variables and SalePrice
correlations = {}
for var in quantitative:
    correlations[var] = df_train['SalePrice'].corr(df_train[var], method='spearman')

# Creating a DataFrame to store correlations
correlation_df = pd.DataFrame({'Variable': list(correlations.keys()), 'Spearman Correlation': list(correlations.values())})

# Sorting correlations in descending order
correlation_df = correlation_df.sort_values(by='Spearman Correlation', ascending=False)

# Creating a bar plot of correlations with improved grid and custom colors
plt.figure(figsize=(10, 12))
ax = sns.barplot(data=correlation_df, x='Spearman Correlation', y='Variable', palette='coolwarm_r')
plt.title('Spearman Correlations with SalePrice')
plt.xlabel('Spearman Correlation')
plt.ylabel('Variable')

# Adding a grid
ax.xaxis.grid(True, linestyle='--', alpha=0.6)
ax.set_axisbelow(True)

# Adjusting the size of variable names
plt.xticks(fontsize=12)  # You can adjust the font size as needed

plt.show()

In the literature ([reference link](https://www.jstor.org/stable/pdf/1631943.pdf)), it is observed that both positively and negatively monotonic values can be significantly correlated from -0.5, indicating a "low" correlation, to 0.5, indicating a "high" correlation. Therefore, let's identify the values that are more correlated and filter them based on the columns with NaN values.

**Reference**

Wissler, C. (1905). The Spearman correlation formula. Science, 22(558), 309-311.

In [ ]:
# Interval Spearman Corr
significant_results = correlation_df[(correlation_df['Spearman Correlation'] > 0.5) | (correlation_df['Spearman Correlation'] < -0.5)]

#Get the names
significant_cols = significant_results['Variable'].tolist()

# Find the variables that match the names in 'significant_cols'
matching_variables = nan_columns[nan_columns.index.isin(significant_cols)]

cols_to_fix_quanti = list(matching_variables.index)

print("Among the missing values that are statistically significant for the research, the only found variable is 'Year garage was built':")
print()
variables_to_fix = '\n'.join(cols_to_fix_quanti)
print(f"{variables_to_fix}")



**Conclusion**

According to my favorite philosopher, Garfield, he says, "Why do tomorrow what you can do a month from now?" Are we obviously starting with the simplest and then moving on to the more difficult? First, I predict the numerical values, and then I classify the categorical variables? Does the order of factors alter the product? 

Let's consider that, if I develop a prediction model for my lone numerical variable, I need to take into account all the other variables in the DataFrame to check their relevance, just as I did for SalePrice. In this context, I can try to replace the categorical values, at least some of them (I promise I will try), starting with the basics, simply by analyzing their context and seeing how I can fill in the gaps depending on the extent of the missing values. In this case, I will use logic before harnessing the power of any Machine Learning model.

### Feature Enginer

Don't forget it!

   - FireplaceQu
   - GarageCond
   - GarageType
   - GarageFinish
   - GarageQual
   - BsmtExposure
   - BsmtQual
   - BsmtCond
   - BsmtFinType1
   - MasVnrType
   - Electrical

**Electrical Sys**

Several factors influence the electrical system of a residence. However, these "endogenous" factors, internal ones like wiring, grounding, etc., we do not have as information. However, we do know that the type of electrical system is directly related to the consumption pattern. For example, the use of air conditioning, housing zone, whether it's for agriculture, commerce, urban areas, type of housing, etc. Understanding electrical systems:

Electrical System (Electrical):

   - Standard Circuit Breakers & Romex (SBrkr): This is a modern and standard electrical system with circuit breakers and Romex wiring. It is suitable for most single-family homes and conventional residences.

   - Fuse Box over 60 AMP and all Romex wiring (FuseA): A fuse box system with Romex wiring, typically suitable for older homes with higher electrical capacity.

   - 60 AMP Fuse Box and mostly Romex wiring (FuseF): A fuse box system with limited electrical capacity, suitable for older homes with lower electrical demand.

   - 60 AMP Fuse Box and mostly knob & tube wiring (FuseP): This electrical system is older and may be less safe. It is important to assess and update these systems to meet modern standards.

   - Mixed (Mix): A mixed electrical system may indicate that different parts of the residence have different types of wiring and electrical capacity. It may be necessary to assess and standardize the electrical system.

In [ ]:
df_all['Electrical'].value_counts()

I selected some features to look: 
    
   - General conditions of the house (we assume that this characteristic is an implicit attribute of the electrical system) 
   - Use of air conditioning
   - Miscellaneous features (we understand that if a house includes an elevator or has two garages, it likely indicates a robust electrical system, or am I entirely mistaken?), and...
   - The geographical location (in commercial and industrial areas, the likelihood of an outdated electrical system might be lower due to safety concerns—unless it's Tyler Durden's house ???).
   - Do you receive all the sanitation and energy services?
   - Foundation -- Wood?

In [ ]:
miss_electrical = df_all[df_all['Electrical'] == 'Missing_Value']
miss_electrical[['OverallCond', 'CentralAir', 'MiscFeature', 'MSZoning', 'Utilities', 'Foundation']]

We can preemptively deduce that the electrical system is within the database. Despite its moderate rating, the property is situated in a sparsely populated urban area, features air conditioning, and is serviced by all public utilities, including sanitation and electricity.

In [ ]:
df_all['Electrical'] = df_all['Electrical'].replace('Missing_Value', df_all['Electrical'].mode()[0])

**MasVnrType**

MasVnrType: Masonry veneer type

  - BrkCmn	Brick Common
  - BrkFace	Brick Face
  - CBlock	Cinder Block
  - None	None
  - Stone	Stone
       
The "MasVnrType" (Masonry Veneer Type) variable can be influenced by several factors, including:
   - Overall House Condition (OverallQual): The overall condition and quality of the house may affect the choice of masonry veneer type. A higher-quality house may be more likely to have premium veneer options like stone.
   - House Style (HouseStyle): Different house styles may be associated with specific masonry veneer types. For example, a traditional or colonial-style house may be more likely to feature brick veneer.
   - Neighborhood (Neighborhood): The neighborhood in which the house is located can influence the choice of masonry veneer type. Homes in upscale neighborhoods may opt for stone veneer for a more luxurious appearance.
   - Year Built (YearBuilt): Older homes may have different masonry veneer types compared to newer homes. Historic homes might feature traditional brick veneer.
   - Exterior Quality (ExterQual): The quality of the exterior materials may determine the masonry veneer type. A house with a high-quality exterior may feature premium veneer options.
   - Lot Size (LotArea): Larger lots may allow for more flexibility in choosing masonry veneer types. Stone veneer, for example, might be more common in houses with spacious lots.
   - Architectural Style (BldgType): Different types of dwellings may have preferences for specific masonry veneer types. For example, single-family homes (1Fam) and townhouses (TwnhsE, TwnhsI) may have varying veneer choices.


In [ ]:
df_all['MasVnrType'].value_counts()

In [ ]:
miss_masvnrt = df_all[df_all['MasVnrType'] == 'Missing_Value']
miss_masvnrt[['OverallQual','HouseStyle','Neighborhood', 'YearBuilt', 'ExterQual', 'LotArea', 'BldgType']]

In [ ]:
result_year_a = df_all.loc[df_all['YearBuilt'] < 1960, ['YearBuilt', 'MasVnrType']]
result_year_b = df_all.loc[(df_all['YearBuilt'] < 1960), ['YearBuilt', 'MasVnrType', 'Neighborhood']]
result_year_b = result_year_b[result_year_b['Neighborhood'] == 'Crawfor']

print("Value Counts for MasVnrType in result_year_a:")
print(result_year_a['MasVnrType'].value_counts())

print("\nValue Counts for MasVnrType in result_year_b:")
print(result_year_b['MasVnrType'].value_counts())

### DecisionTreeClassifier to fill NaN values

**I will estimate this model to fill in both the NaN values of the qualitative and quantitative variables**

To identify correlated variables among the features with missing values that need to be imputed, we can leverage their qualitative nature by ranking them based on Kendall's correlation, as described earlier.

In [ ]:
from scipy.stats import kendalltau

def find_highly_correlated_variables(df_train, variables_of_interest, threshold=0.7):
    correlated_variables_dict = {}
    
    for var in variables_of_interest:
        correlations = []
        for column in df_train.columns:
            if column != var:
                correlation, _ = kendalltau(df_train[var], df_train[column])
                correlations.append((column, correlation))

        highly_correlated = [(col, corr) for col, corr in correlations if abs(corr) >= threshold]

        highly_correlated.sort(key=lambda x: abs(x[1]), reverse=True)

        correlated_variables_dict[var] = highly_correlated

    result_df = pd.DataFrame(columns=['Target', 'Feature', 'Kendall Cor'])

    for var, correlated_vars in correlated_variables_dict.items():
        df = pd.DataFrame({'Target': var, 'Feature': [col for col, _ in correlated_vars], 'Kendall Cor': [corr for _, corr in correlated_vars]})
        result_df = pd.concat([result_df, df], ignore_index=True)

    return result_df

In [ ]:
variables_of_interest = ['FireplaceQu', 
                         'GarageCond', 
                         'GarageType', 
                         'GarageFinish', 
                         'GarageQual', 
                         'BsmtExposure', 
                         'BsmtQual', 
                         'BsmtCond', 
                         'BsmtFinType1',
                         'BsmtCond',
                         'BsmtFinType1',
                         'MasVnrType',
                        ]

correlation_results = find_highly_correlated_variables(df_all, variables_of_interest, threshold=0.35)

# Display the results table
print(correlation_results)

Where to start? The missing correlations are specifically associated with variables that exhibit high rates of missing values, such as FireplaceQu and BsmtExposure (the latter seemingly demonstrating an indifference or lack of association with other variables). 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.compose import ColumnTransformer

def train_and_impute(df_train, correlation_results, qualitative, quantitative, mode="classification"):
    # Check unique values in the "Target" column
    unique_targets = correlation_results['Target'].unique()

    for target in unique_targets:
        # Identify rows with missing values in the target column
        missing_rows = df_train[df_train[target] == 'Missing_Value']
        non_missing_rows = df_train[df_train[target] != 'Missing_Value']

        # Filter the original DataFrame based on the current target value
        features = correlation_results[correlation_results['Target'] == target].drop(columns=['Target', 'Kendall Cor'])
        features = features['Feature'].to_list()

        # Separate the features (X) and target labels (y) for non-missing values
        X = non_missing_rows[features]
        y = non_missing_rows[target]

        # Combine the missing rows with non-missing rows for encoding
        combined_rows = pd.concat([X, missing_rows[features]])

        # Split Quali and Quanti to transform
        quali = [var for var in features if var in qualitative]
        quanti = [var for var in features if var in quantitative]

        # Apply OneHotEncoding
        if len(quali) > 0:
            ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), quali)], remainder='passthrough')
            X_encoded = ct.fit_transform(combined_rows[quali])
            X_encoded = X_encoded.toarray()
        else:
            pass

        if len(quanti) > 0:
            ss = StandardScaler()
            X_quanti_scaled = ss.fit_transform(combined_rows[quanti])
            X_encoded = np.hstack((X_encoded, X_quanti_scaled))
        else:
            X_encoded = X_encoded

        # Label Encoder to target for non-missing values
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)

        # Split into train (80%) and test (20%) set for non-missing values
        X_train, X_test, y_train, y_test = train_test_split(X_encoded[:len(y_encoded)], y_encoded, test_size=0.2, random_state=42)

        # Create a Decision Tree Classifier or Regressor based on the mode
        if mode == "classification":
            model = DecisionTreeClassifier(random_state=42)
        elif mode == "regression":
            model = DecisionTreeRegressor(random_state=42)
        else:
            raise ValueError("Mode must be 'classification' or 'regression'.")

        # Train the model
        model.fit(X_train, y_train)

        # Predict for rows with 'Missing_Value' in the target column
        X_missing_encoded = X_encoded[len(y_encoded):]
        if mode == "classification":
            y_pred_missing = model.predict(X_missing_encoded)
        elif mode == "regression":
            y_pred_missing = model.predict(X_missing_encoded).astype(int)

        # Update the original DataFrame with predictions
        df_train.loc[df_train[target] == 'Missing_Value', target] = label_encoder.inverse_transform(y_pred_missing)

        # Calculate accuracy or mean squared error based on the mode
        if mode == "classification":
            y_pred_test = model.predict(X_test)
            accuracy_test = accuracy_score(y_test, y_pred_test)
            print(f'Model accuracy on test data for {target}: {accuracy_test:.2f}')
        elif mode == "regression":
            y_pred_test = model.predict(X_test)
            mse_test = mean_squared_error(y_test, y_pred_test)
            print(f'Mean Squared Error on test data for {target}: {mse_test:.2f}')


In [ ]:
train_and_impute(df_train, correlation_results, qualitative, quantitative, mode="classification")

In [ ]:
columns_with_missing_values = df_all.columns[df_all.eq('Missing_Value').any()]
print("Colunas com valores 'Missing_Value':")
print(columns_with_missing_values)

## Numerical col 'GarageYrBlt'

In [ ]:
df_all_temp = df_all.drop(columns=['SalePrice'])

#Split missing from non missing in GYB
non_missing_gyb = df_all_temp[df_all_temp['GarageYrBlt'].isnull() == False]

# Verifique se há valores ausentes em colunas numéricas
numeric_columns_with_missing = df_all_temp.select_dtypes(include='number').columns[df_all_temp.select_dtypes(include='number').isnull().any()]

# Exiba as colunas numéricas com valores ausentes, se houver
if not numeric_columns_with_missing.empty:
    print(f"Colunas numéricas com valores ausentes: {', '.join(numeric_columns_with_missing)}")
else:
    print("Não há valores ausentes em colunas numéricas.")

#Check correlation 
#result = find_highly_correlated_variables(non_missing_gyb, ['GarageYrBlt'], threshold = 0.4)

#df_all['GarageYrBlt'].fillna('Missing_Value', inplace=True)

#result
#for col in quantitative:
 #   df_all[col].fillna(, inplace=True) #This Missing_Value marker for the numeric variables helps the regression function to identify what is non-missing from what is missing
    
#train_and_impute(df_all, result, qualitative, quantitative, mode="regression")

In [ ]:
# Verifique quais variáveis possuem valores ausentes
variables_with_missing_values = df_all.columns[df_all.isnull().any()].tolist()

# Exiba as variáveis com valores ausentes
print("Variables with missing values:")
for variable in variables_with_missing_values:
    missing_count = df_all[variable].isnull().sum()
    print(f"{variable}: {missing_count} missing values")

# Modelling Regression

## Split dataset in train set and test set 

In [ ]:
#Qualitative: Nominal variables 

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), nominal_variables)], remainder='passthrough')
X_nominal_encoded = ct.fit_transform(df_train[nominal_variables])
X_nominal_encoded = X_nominal_encoded.toarray()

In [ ]:
#Qualitative: Ordinal variables

# Create a copy of the DataFrame with only the columns you want to encode
df_ordinal = df_train[ordinal_variables].copy()

# Create a dictionary to store the LabelEncoders for each column
label_encoders = {}

# Iterate over the columns and apply the LabelEncoder
for col in ordinal_variables:
    label_encoder = LabelEncoder()
    df_ordinal[col] = label_encoder.fit_transform(df_ordinal[col])
    label_encoders[col] = label_encoder

X_ordinal_encoded = df_ordinal.values

# Now, df_ordinal contains the encoded columns, and label_encoders contains the encoders for each column.

#If I wanna reverse:
# Iterate over the columns and apply the inverse_transform to decode the values
# for col in ordinal_variables:
#   label_encoder = label_encoders[col]
#  df_decoded[col] = label_encoder.inverse_transform(df_decoded[col])
# Now, df_decoded contains the original values before encoding.

In [ ]:
# Horizontal concat
X = np.hstack((X_nominal_encoded, X_ordinal_encoded))
print(X.shape)

In [ ]:
y = df_train['SalePrice'].values

In [ ]:
# Splitting the dataset into the Training set and Test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Select Model

### Ridge and Lasso regressions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Suponha que você tenha um conjunto de dados com features (X) e target (y)
# Substitua os valores abaixo com seus próprios dados
X = np.random.rand(100, 10)  # Exemplo de features
y = np.random.rand(100)  # Exemplo de target

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
    
# Defina os modelos
models = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ANN': MLPRegressor(max_iter=1000),  # Redes Neurais Artificiais (MLP)
    'XGBoost': XGBRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'RandomForest': RandomForestRegressor()
}

# Realize a validação cruzada para cada modelo
# Realize a validação cruzada para cada modelo e armazene o RMSE médio em um dicionário
model_rmse = {}

for model_name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    
    # Calcule a média dos RMSE da validação cruzada
    mean_rmse = np.mean(rmse_scores)
    model_rmse[model_name] = mean_rmse
    
    print(f"Modelo: {model_name}")
    print(f"RMSE médio da validação cruzada: {mean_rmse:.4f}\n")

# Escolha o modelo com o menor RMSE médio na validação cruzada
best_model_name = min(model_rmse, key=model_rmse.get)
best_model = models[best_model_name]

# Treine o modelo selecionado no conjunto de treinamento completo
best_model.fit(X_train, y_train)

# Preveja os valores no conjunto de teste
y_pred = best_model.predict(X_test)

# Calcule o RMSE no conjunto de teste
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Melhor modelo ({best_model_name}): RMSE no conjunto de teste: {test_rmse:.4f}\n")

# Train the three best models for plotting predictions
sorted_models = sorted(models.items(), key=lambda x: model_rmse[x[0]])
top_3_models = [models[model[0]] for model in sorted_models[:3]]

# Generate predictions from the three best models
predictions = []

for model in top_3_models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions.append(y_pred)

# Plotar previsões dos três melhores modelos em gráficos separados
for i, model_name in enumerate([model[0] for model in sorted_models[:3]]):
    plt.figure(figsize=(8, 6))
    plt.title(f'Previsões - Modelo {model_name}', fontsize=16)
    plt.xlabel('Exemplo', fontsize=14)
    plt.ylabel('Valor Real vs. Valor Previsto', fontsize=14)
    
    plt.plot(y_test, label='Real', linestyle='--')
    plt.plot(predictions[i], label='Previsto')
    
    plt.legend()
    plt.show()

In [ ]:
df_train.shape, df_all.shape